This Jupyter notebook uses the code from Daniel Corey and Dante Luber used for the paper [Singular matroid realization spaces](https://arxiv.org/abs/2307.11915) to compute realization spaces for $(3,11)$-matroids, which are matroids of rank 3 and ground set size 11. Daniel Corey and Dante Luber also provide the simple $(3,11)$-matroids that satisfies the 3 lines property, meaning each element of the ground set is contained in at least 3 lines. Out of the simple $(3,11)$-matroids with 3 lines property, they also provide which ones are not realizable over $\mathbb{C}$ and which ones with a maximal circuit such that ideal of the realization space are zero, univariate, or multivariate and principal. All the $(3,11)$-matroids with the 3 lines property fall into one of those categories.

Out of the 232928 simple $(3,11)$-matroids, 16234 satisfy the 3 lines property. Out of the 16234 that satisfy the 3 lines property, 4718 are not realizable over $\mathbb{C}$. Then, out of the remaining 11516 simple $(3,11)$-matroids that satisfy, it was determined that there are maximal circuits for each matroid such that
- 2220 of them have the zero ideal for the realization space,
- 3658 of them have a univariate ideal for the realization space, and
- 5638 of them have a multivariate, principal ideal for the realization space (and there are no maximal circuits such that the ideal of the realization space is univariate).

# Importing realization space code

First, we clone the [Github repository](https://github.com/dcorey2814/matroidRealizationSpaces/tree/main) from Daniel Corey and Dante Luber. The code uses Julia and the package OSCAR.

In [ ]:
!git clone https://github.com/dcorey2814/matroidRealizationSpaces.git

Cloning into 'matroidRealizationSpaces'...
remote: Enumerating objects: 1221, done.
remote: Counting objects: 100% (388/388), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 1221 (delta 224), reused 287 (delta 138), pack-reused 833
Receiving objects: 100% (1221/1221), 197.64 MiB | 23.62 MiB/s, done.
Resolving deltas: 100% (775/775), done.


# Importing the necessary packages

In [ ]:
using Oscar

In [ ]:
cd("matroidRealizationSpaces")

In [ ]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))


realization_space_2_singular_locus (generic function with 1 method)

# Getting univariate polynomials

The file `univariate_ideal_3_11.dat` contains all simple $(3,11)$-matroids with the 3 lines property such that there exists a maximal circuit such that the ideal of the realization space is univariate (and principal). The following code extracts a polynomial that generates the ideal of the realization space of each of these matroids and places these polynomials in `univariate_polys.txt`.

In [ ]:
univariate_ideal = vec(readlines(joinpath(currentDir, "d3n11/data/univariate_ideal_3_11.dat")));
length(univariate_ideal)

3658

Following cell took 28 minutes to run.

In [ ]:
file_path = "../data/univariate_polys.txt"  # Replace this with the actual path to your file
file = open(file_path, "w")

for cir_Qstr in univariate_ideal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11)
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)

    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    vs = ideal_vars(gens(I))
    length(vs) != 1 && error("not univariate")
    f = gens(I)[1]
    write(file, string(f) * "\n")
end

close(file)

# Getting multivariate polynomials

In this section, we use the data from `multivariate_principal_ideal_3_11.dat` to get all the simple $(3,11)$-matroids with the 3 lines property such that there exists a maximal circuit so that the ideal of the realization space is multivariate and principal. This data does not include the univariate cases from the previous section. The following code extracts a multivariate polynomial that generates the ideal of the realization space and places these polynomials in `multivariate_polys.txt`.

In [ ]:
multivariate_principal = vec(readlines(joinpath(currentDir, "d3n11/data/multivariate_principal_ideal_3_11.dat")));
length(multivariate_principal)

5638

In [ ]:
file_path = "../data/multivariate_polys.txt"  # Replace this with the actual path to your file
file = open(file_path, "w")
for cir_Qstr in multivariate_principal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11)
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)

    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal")
    f = Igens[1]
    write(file, string(f) * "\n")
end

close(file)